# ELAS — Validation complète avec jeux réels (Pantheon+SH0ES, BAO, CMB)

**Objectif** : recalcul multi-sondes (SNe Ia, BAO, CMB) avec vos *vrais* fichiers.
Le notebook accepte différents formats réels et harmonise les colonnes automatiquement.

- **SN** : Pantheon+SH0ES (z, μ, σ_μ) ou colonnes officielles (`zHD`, `MU`, `MUERR`, `m_b_corr`, `m_b_corr_err`).
- **BAO** : compilation (z_eff, type, value, sigma) avec `type ∈ {DM_over_rd, DH_over_rd, DV_over_rd, rs_over_DV}`.
- **CMB** : distance priors Planck 2018 — deux CSV (`means.csv`, `inv_cov.csv`) avec paramètres (`R`, `l_a`, `omega_b_h2`).

Par défaut, **H0 = 73 km/s/Mpc** (cohérent SH0ES). Vous pouvez changer H0 si besoin.

## 0) Imports, constantes, utilitaires

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
c = 299792.458; H0 = 73.0; Omega_m = 0.315; Omega_b_h2 = 0.02237
ELAS_DEFAULT = dict(delta=0.05, Omega_osc=1.5, phi=-2.62)
try:
    from scipy.integrate import cumulative_trapezoid
except Exception:
    def cumulative_trapezoid(y, x, initial=0.0):
        y = np.asarray(y, dtype=float); x = np.asarray(x, dtype=float)
        trap = 0.5*(y[1:]+y[:-1])*(x[1:]-x[:-1])
        return np.concatenate(([initial], np.cumsum(trap)))
def quick_info(df, name):
    print(f"\n{name}: {df.shape[0]} lignes, colonnes = {list(df.columns)}"); print(df.head())

## 1) Modèles (ΛCDM / ELAS) et distances (intégration 0→z)

In [2]:
def E_LCDM(z, H0=H0, Omega_m=Omega_m):
    z = np.asarray(z, dtype=float); return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m))
def E_ELAS(z, H0=H0, Omega_m=Omega_m, delta=0.05, Omega_osc=1.5, phi=-2.62):
    z = np.asarray(z, dtype=float)
    return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m)*(1.0 + delta*np.cos(Omega_osc*np.log(1+z) + phi)))
def distances(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z = np.asarray(z, dtype=float); zmax = float(np.max(z)); zz = np.linspace(0.0, max(zmax, 1e-6), 8000)
    EE = E_LCDM(zz, H0, Omega_m) if model.upper()=="LCDM" else E_ELAS(zz, H0, Omega_m, **({**ELAS_DEFAULT, **elas}))
    Dc_grid = (c/H0) * cumulative_trapezoid(1.0/EE, zz, initial=0.0); Dc = np.interp(z, zz, Dc_grid); Dl = (1.0 + z) * Dc
    return dict(Dc=Dc, Dl=Dl)
def mu_theory(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    Dl = distances(z, model, H0, Omega_m, **elas)["Dl"]; Dl = np.clip(Dl, 1e-6, None); return 5.0*np.log10(Dl) + 25.0

## 2) SN : chargement flexible (Pantheon+SH0ES) et χ² avec marginalisation de M

In [5]:
from io import StringIO
def load_sn_flexible():
    import pandas as pd
    try:
        from google.colab import files
        print("➡️ Téléverse votre CSV Pantheon+SH0ES (SN).");
        up = files.upload(); sn_csv = None
        for k in up.keys():
            if k.lower().endswith(".csv"): sn_csv = k; break
    except Exception:
        sn_csv = None
    if sn_csv is None: raise RuntimeError("Aucun fichier SN téléversé.")
    df = pd.read_csv(sn_csv); cols = {c.lower():c for c in df.columns}
    if ("mu" in cols) and ("muerr" in cols or "sigma_mu" in cols):
        zcol = cols.get("z") or cols.get("zhd") or cols.get("zhelio") or list(df.columns)[0]
        muc = cols["mu"]; sigc = cols.get("muerr") or cols["sigma_mu"]
        return df[zcol].astype(float).values, df[muc].astype(float).values, df[sigc].astype(float).values, df
    if ("m_b_corr" in cols) and ("m_b_corr_err" in cols):
        zcol = cols.get("z") or cols.get("zhd") or cols.get("zhelio") or list(df.columns)[0]
        return df[zcol].astype(float).values, df[cols["m_b_corr"]].astype(float).values, df[cols["m_b_corr_err"]].astype(float).values, df
    if ("mu" in cols) and ("dmu" in cols):
        zcol = cols.get("z") or cols.get("zhd") or cols.get("zhelio") or list(df.columns)[0]
        return df[zcol].astype(float).values, df[cols["mu"]].astype(float).values, df[cols["dmu"]].astype(float).values, df
    raise ValueError("Colonnes SN non reconnues.")
def chi2_sn_M_marginalized(z, mu_obs, sigma_mu, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z = np.asarray(z, dtype=float); mu_obs = np.asarray(mu_obs, dtype=float); sigma_mu = np.asarray(sigma_mu, dtype=float)
    mu_th = mu_theory(z, model, H0, Omega_m, **elas); w = 1.0 / (sigma_mu**2)
    deltaM = np.sum(w*(mu_obs - mu_th)) / np.sum(w); resid = mu_obs - (mu_th + deltaM); chi2 = np.sum(w * resid**2)
    return float(chi2), float(deltaM)
z_sn, mu_obs, s_mu, df_sn = load_sn_flexible(); quick_info(df_sn, "SN (brut)")
chi2_sn_LCDM, dM_LCDM = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="LCDM")
chi2_sn_ELAS, dM_ELAS = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="ELAS", **ELAS_DEFAULT)
print("SN χ² (LCDM, ELAS) =", chi2_sn_LCDM, chi2_sn_ELAS); print("Offsets δM (LCDM, ELAS) =", dM_LCDM, dM_ELAS)

➡️ Téléverse votre CSV Pantheon+SH0ES (SN).


Saving Pantheon+SH0ES.dat to Pantheon+SH0ES (1).dat


RuntimeError: Aucun fichier SN téléversé.

In [4]:
# --- Remplace/ajoute cette version tolérante du chargeur SN ---
import pandas as pd
import numpy as np

def load_sn_flexible_any():
    """
    Charge un fichier Pantheon+SH0ES en .dat/.txt/.csv et renvoie (z, mu, sigma_mu, df).
    Gère les entêtes Pantheon+ usuelles: z, zHD, zHEL, MU, MUERR, m_b_corr, m_b_corr_err, etc.
    """
    try:
        from google.colab import files
        print("➡️ Téléverse votre Pantheon+ (csv/dat/txt).")
        up = files.upload()
        sn_path = None
        for k in up.keys():
            if k.lower().endswith((".csv", ".dat", ".txt")):
                sn_path = k; break
    except Exception:
        sn_path = None

    if sn_path is None:
        raise RuntimeError("Aucun fichier SN téléversé.")

    # Essais robustes de lecture: CSV, puis espace/onglet (delim auto)
    try:
        df = pd.read_csv(sn_path)
    except Exception:
        df = pd.read_csv(sn_path, sep=r"\s+", engine="python", comment="#")

    # Normaliser les noms en minuscule pour la détection
    cols_lower = {c.lower(): c for c in df.columns}

    # Chercher une colonne de redshift
    z_key = None
    for cand in ["z", "zhd", "zhelio", "zhel", "zcmb", "z_cmb", "zcmb", "zobs", "z_obs"]:
        if cand in cols_lower:
            z_key = cols_lower[cand]; break
    if z_key is None:
        # Si le 1er champ ressemble à z
        z_key = df.columns[0]

    # 3 cas principaux pour mu / sigma_mu
    mu_key = sig_key = None

    # Cas direct: mu + (muerr ou sigma_mu)
    if ("mu" in cols_lower) and (("muerr" in cols_lower) or ("sigma_mu" in cols_lower) or ("dmu" in cols_lower)):
        mu_key = cols_lower["mu"]
        if "muerr" in cols_lower: sig_key = cols_lower["muerr"]
        elif "sigma_mu" in cols_lower: sig_key = cols_lower["sigma_mu"]
        else: sig_key = cols_lower["dmu"]

    # Cas Pantheon+ m_b_corr, m_b_corr_err (M absorbé par marginalisation de M)
    if mu_key is None and ("m_b_corr" in cols_lower) and ("m_b_corr_err" in cols_lower):
        mu_key  = cols_lower["m_b_corr"]
        sig_key = cols_lower["m_b_corr_err"]

    # Cas “MU, MUERR” (lettres majuscules)
    if mu_key is None and ("mu" not in cols_lower):
        # vérifier versions majuscules (certaines distros ne changent pas la casse)
        for C in df.columns:
            if C.upper() == "MU": mu_key = C
            if C.upper() in ("MUERR", "SIGMA_MU", "DMU"): sig_key = C

    if mu_key is None or sig_key is None:
        raise ValueError(f"Colonnes non reconnues pour μ/σμ. Colonnes trouvées: {list(df.columns)}")

    # Extraire vecteurs numpy
    z = df[z_key].astype(float).values
    mu = df[mu_key].astype(float).values
    s  = df[sig_key].astype(float).values

    # Nettoyage basique
    m = np.isfinite(z) & np.isfinite(mu) & np.isfinite(s) & (s > 0)
    z, mu, s = z[m], mu[m], s[m]
    df_clean = df.loc[m].copy()

    print(f"SN ok: {len(z)} objets — colonnes z='{z_key}', mu='{mu_key}', sigma='{sig_key}'")
    return z, mu, s, df_clean


## 3) BAO : chargement flexible et χ²

In [6]:
def load_bao_flexible():
    try:
        from google.colab import files
        print("➡️ Téléverse votre CSV BAO (z_eff,type,value,sigma).");
        up = files.upload(); bao_csv = None
        for k in up.keys():
            if k.lower().endswith(".csv"): bao_csv = k; break
    except Exception:
        bao_csv = None
    if bao_csv is None:
        print("⚠️ Aucun BAO fourni : χ²_BAO=0."); return None
    df = pd.read_csv(bao_csv); cols = {c.lower():c for c in df.columns}
    for r in ["z_eff","type","value","sigma"]:
        if r not in cols: raise ValueError(f"Colonne requise manquante pour BAO: {r}")
    return df.rename(columns=cols)
def rd_fid(): return 147.09
def bao_theory_value(z, typ, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    d = distances(np.array([z]), model, H0, Omega_m, **elas); Dm = d["Dc"][0]
    Dh = (c/H0)/(E_LCDM if model.upper()=="LCDM" else E_ELAS)(np.array([z]), H0, Omega_m, **({} if model.upper()=="LCDM" else {**ELAS_DEFAULT, **elas}))[0]
    rd = rd_fid()
    if typ=="DM_over_rd": return Dm/rd
    if typ=="DH_over_rd": return Dh/rd
    if typ=="DV_over_rd": return (z*Dh*Dm*Dm)**(1/3.0)/rd
    if typ=="rs_over_DV": return rd/((z*Dh*Dm*Dm)**(1/3.0))
    raise ValueError("type BAO inconnu")
def chi2_bao(df, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    if df is None or len(df)==0: return 0.0
    s = 0.0
    for _, r in df.iterrows():
        z, val, sig, typ = float(r["z_eff"]), float(r["value"]), float(r["sigma"]), str(r["type"])
        th = bao_theory_value(z, typ, model, H0, Omega_m, **elas); s += ((th - val)/sig)**2
    return float(s)
df_bao = load_bao_flexible()
if df_bao is not None: quick_info(df_bao, "BAO (brut)")
chi2_bao_LCDM = chi2_bao(df_bao, "LCDM"); chi2_bao_ELAS = chi2_bao(df_bao, "ELAS", **ELAS_DEFAULT)
print("BAO χ² (LCDM, ELAS) =", chi2_bao_LCDM, chi2_bao_ELAS)

➡️ Téléverse votre CSV BAO (z_eff,type,value,sigma).


Saving BAO.md to BAO.md
⚠️ Aucun BAO fourni : χ²_BAO=0.
BAO χ² (LCDM, ELAS) = 0.0 0.0


## 4) CMB : distance priors (Planck 2018) — chargement flexible

In [7]:
def load_cmb_distance_priors():
    try:
        from google.colab import files
        print("➡️ Téléverse means.csv puis inv_cov.csv (CMB distance priors).");
        up = files.upload(); means_csv = inv_csv = None
        for k in up.keys():
            if "means" in k and k.endswith(".csv"): means_csv = k
            if "inv_cov" in k and k.endswith(".csv"): inv_csv = k
    except Exception:
        means_csv = inv_csv = None
    if means_csv is None or inv_csv is None:
        print("⚠️ Aucun CMB fourni : χ²_CMB=0."); return None, None
    return pd.read_csv(means_csv), pd.read_csv(inv_csv, index_col=0)
def cmb_distance_priors_theory(H0=H0, Omega_m=Omega_m, Omega_b_h2=Omega_b_h2):
    zstar = 1089.0; zgrid = np.linspace(0, zstar, 12000)
    Dc = (c/H0)*np.trapz(1.0/E_LCDM(zgrid, H0, Omega_m), zgrid); rs = 147.09
    R  = np.sqrt(Omega_m)*(H0/100.0) * (Dc/(c/100.0)); la = np.pi * (Dc/rs)
    return float(R), float(la), float(Omega_b_h2)
def chi2_cmb_from_uploaded(means_df, icov_df, H0=H0, Omega_m=Omega_m):
    if means_df is None or icov_df is None: return 0.0
    order = ["R","l_a","omega_b_h2"]; vec_th = np.array(cmb_distance_priors_theory(H0, Omega_m, Omega_b_h2))
    vec_obs = means_df.set_index("parameter").loc[order,"mean"].values; icov = icov_df.loc[order, order].values; d = vec_th - vec_obs
    return float(d @ icov @ d)
cmb_means, cmb_icov = load_cmb_distance_priors()
chi2_cmb_LCDM = chi2_cmb_from_uploaded(cmb_means, cmb_icov); chi2_cmb_ELAS = chi2_cmb_from_uploaded(cmb_means, cmb_icov)
print("CMB χ² (LCDM, ELAS) =", chi2_cmb_LCDM, chi2_cmb_ELAS)

➡️ Téléverse means.csv puis inv_cov.csv (CMB distance priors).


Saving inv_cov.csv to inv_cov.csv
⚠️ Aucun CMB fourni : χ²_CMB=0.
CMB χ² (LCDM, ELAS) = 0.0 0.0


## 5) Totaux + Δχ², AIC, BIC + export

In [8]:
tot_LCDM = chi2_sn_LCDM + chi2_bao_LCDM + chi2_cmb_LCDM
tot_ELAS = chi2_sn_ELAS + chi2_bao_ELAS + chi2_cmb_ELAS
k_LCDM, k_ELAS = 0, 3
N_eff = len(z_sn) + (0 if df_bao is None else len(df_bao)) + (0 if cmb_means is None else 3)
dchi2 = tot_LCDM - tot_ELAS; dAIC = dchi2 - 2*(k_ELAS - k_LCDM); import numpy as np
dBIC = dchi2 - (k_ELAS - k_LCDM)*np.log(max(N_eff,1))
summary = pd.DataFrame([
    dict(model="LCDM", H0=H0, Omega_m=Omega_m, chi2_sn=chi2_sn_LCDM, chi2_bao=chi2_bao_LCDM, chi2_cmb=chi2_cmb_LCDM, chi2_total=tot_LCDM),
    dict(model="ELAS", H0=H0, Omega_m=Omega_m, delta=ELAS_DEFAULT["delta"], Omega_osc=ELAS_DEFAULT["Omega_osc"], phi=ELAS_DEFAULT["phi"],
         chi2_sn=chi2_sn_ELAS, chi2_bao=chi2_bao_ELAS, chi2_cmb=chi2_cmb_ELAS, chi2_total=tot_ELAS)
]); print("=== RÉSUMÉ TOTAL (SN+BAO+CMB) ==="); print(summary)
print(f"\nΔχ² (LCDM − ELAS) = {dchi2}"); print(f"ΔAIC = {dAIC}  ΔBIC = {dBIC}   (N_eff = {N_eff} )")
summary.to_csv("ELAS_full_validation_summary.csv", index=False)

NameError: name 'chi2_sn_LCDM' is not defined

## 6) Scan contraint (SN-only) — δ ≤ 0.15, 0.5 ≤ Ω ≤ 2.5

In [ ]:
grid_delta = np.linspace(0.0, 0.15, 16); grid_Omega = np.linspace(0.5, 2.5, 21); grid_phi = np.linspace(-np.pi, np.pi, 121)
best = dict(chi2=np.inf, delta=None, Omega=None, phi=None)
for d in grid_delta:
    for Om in grid_Omega:
        chi2_min = np.inf; phi_min = None
        for ph in grid_phi:
            chi2, _ = chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="ELAS", delta=d, Omega_osc=Om, phi=ph)
            if chi2 < chi2_min: chi2_min, phi_min = chi2, ph
        if chi2_min < best["chi2"]: best.update(dict(chi2=chi2_min, delta=d, Omega=Om, phi=phi_min))
dchi2_best = chi2_sn_LCDM - best["chi2"]
print("=== SCAN CONTRAINT (SN-only) ==="); print("Meilleur ELAS:", best); print(f"Δχ² (LCDM − ELAS_best) = {dchi2_best:.3f}")
pd.DataFrame([best]).to_csv("ELAS_best_SNonly_constrained_full.csv", index=False)

## 7) Exports (figures + CSV)

In [ ]:
zgrid = np.linspace(1e-3, 2.3, 1000)
muE = mu_theory(zgrid, "ELAS", **ELAS_DEFAULT); muL = mu_theory(zgrid, "LCDM"); dmu = muE - muL
plt.figure(figsize=(6,3.5)); plt.plot(zgrid, dmu, lw=2); plt.axhline(0, ls="--", alpha=0.5)
plt.xlabel("z"); plt.ylabel("Δμ (mag)"); plt.title("Δμ(z) = μ_ELAS − μ_LCDM  (H0 = %.1f)" % H0)
plt.tight_layout(); plt.savefig("ELAS_delta_mu_full.png", dpi=160); plt.show()
pd.DataFrame({"z":zgrid, "delta_mu_mag":dmu}).to_csv("ELAS_delta_mu_full.csv", index=False)
print("➡️ Exports : ELAS_full_validation_summary.csv, ELAS_best_SNonly_constrained_full.csv, ELAS_delta_mu_full.csv, ELAS_delta_mu_full.png")